# Atividade de Laboratório 1
### Integrantes
- Leonardo Mallmann
- Caetano Muller
- Euzébio Hensel

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.error import URLError, HTTPError, ContentTooShortError
import re
import csv

In [ ]:
def download(url, user_agent='wswp', charset='utf-8'):
  print('Downloading:', url)
  request = urllib.request.Request(url)
  request.add_header('User-agent', user_agent)
  try:
    resp = urllib.request.urlopen(request)
    cs = resp.headers.get_content_charset()
    if not cs:
        cs = charset
    html = resp.read().decode(cs)
  except (URLError, HTTPError, ContentTooShortError) as e:
    print('Download error:', e.reason)
  return html

In [ ]:
def scrap(html_content):
  soup = BeautifulSoup(html_content, 'html5lib')
  rows = soup.select('table tbody tr')
  tiempo = datetime.now()
  field_names = []
  field_values = []
  for row in rows:
    c = row.select('.w2p_fl label');
    field_names.append(c[0].string.strip().replace(':', ''));
    field_names.append('Date/Time')
    v = row.select('.w2p_fw');
    if not v[0].string:
      if len(re.findall('<img', str(v[0]))) != 0:
        img_src = v[0].select('img')[0];
        field_values.append(img_src['src'])
        continue
      if len(re.findall('<div', str(v[0]))) != 0:
        a_links = v[0].select('div a');
        neighbours = [a_tag.string.strip() for a_tag in a_links]
        field_values.append(neighbours);
        continue
      field_values.append('None')
      continue
    if v[0].string == ' ':
      field_values.append('None')
      continue
    field_values.append(v[0].string)
  field_values.append(tiempo)
  
  return field_values, field_names
    

### Exemplo de inserção no CSV

In [ ]:
def insert_headers(fieldnames):
  with open('places.csv', 'w', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    writer.writeheader()

def insert_data(data_row, fieldnames):
  with open('places.csv', 'a', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    
    writer.writerow({
      f_name: f_value
      for f_name, f_value in zip(fieldnames,data_row)
    }) # or dict(zip(keys, values))

In [ ]:
with open('places.csv', 'r', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    row['Capital'].replace(row['Capital'], 'Lego')
    print(row['Capital'])

In [ ]:
def crawl_sitemap(url):
  sitemap = download(url)
  links = re.findall('\d(\/.*)<\/loc>', sitemap)
  for index, link in enumerate(links):
    html = download(f'http://localhost:8000{link}')
    f_values, f_names = scrap(html)
    if index==0:
      insert_headers(f_names)
    insert_data(f_values, f_names)
    if index==4:
      break

In [ ]:
crawl_sitemap('http://localhost:8000/places/static/sitemap.xml');

In [26]:
def page_monitor(url, csv_path):
    sitemap = download(url)
    lcsv = list(csv.reader(open(csv_path)))
    links = re.findall('\d(\/.*)<\/loc>', sitemap)
    for index, link in enumerate(links):
        html = download(f'http://localhost:8000{link}')
        f_values, f_names = scrap(html)
        for i, j in range(f_values), range(f_names):
            if i!=0 and j!=range(j)-1:
                if lcsv[i][j] != f_values[j]:
                    lcsv[i][j] = f_values[j]
        if index==2:
            break
    writer = csv.writer(open(csv_path, 'w'))
    writer.writerows(lcsv)
        
    
page_monitor("kjsd", 'places.csv')

Downloading: kjsd


ValueError: unknown url type: 'kjsd'